# Two stage miller OTA Sizing Script
Reference: 
- CMOS Analog Circuit Design, Phillip E. Allen
- gm/ID, Boris Murman

In [3]:
from pygmid import Lookup as lk
import numpy as np

In [ ]:
#Specs
gain_bw = 10e6 #minimum gain bandwidth
gain_db = 20 #minimum gain 20 dB
phase_margin = 90 #minimum phase margin 90 deg
c_load = 30e-12 # load capacitance

#C Miller Compensation
c_miller = 0.22 * c_load

#Input Stage
gm_1 = None
ro_1 = None

#Ouptu Stage
gm_2 = None
ro_2 = None

#Miller Compensation

#System
dc_gain = None

#===Transistor Level===
#Differential Pairs



In [1]:
def get_w(mos_type, gm, gm_id_spec, L_spec, vds, title='specs'):
    
    #Define the MOS type
    if mos_type == 0:
        mos_file = 'nfet_03v3.mat'
    elif mos_type == 1:
        mos_file = 'pfet_03v3.mat'
    else:
        print(f"MOS type: 0: NMOS 3.3V, 1: PMOS 3.3")
        return
    
    # the gm_gds we look up and calculate gds from it
    mos = lk(mos_file)
    vgs = mos.look_upVGS(GM_ID=gm_id_spec, L=L_spec, VDS=vds, VSB=0.0)
    vgs = mos.look_upVGS(GM_ID=gm_id_spec, L=L_spec, VDS=vgs, VSB=0.0)
    print('V_GS =', round(float(vgs), 3), 'V')

    # the gm_gds we look up and calculate gds from it
    gm_gds = mos.lookup('GM_GDS', GM_ID=gm_id_spec, L=L_spec, VDS=vgs, VSB=0)
    gds = gm / gm_gds
    print('gds =', round(gds/1e-9, 2), 'nS')

    # find f_T (which is not stored directly, but we can find the gm to gate capacitance ratio)
    gm_cgg = mos.lookup('GM_CGG', GM_ID=gm_id_spec, L=L_spec, VDS=vgs, VSB=0)
    f_T = gm_cgg / (2*np.pi)
    print('Cgg =', round(gm/gm_cgg/1e-15, 1), 'fF')
    print('f_T =', round(f_T/1e6, 1), 'MHz')

    # find the W of the diode transistor
    id_spec = gm/gm_id_spec
    print('Id =', round(id_spec*1000000, 2), 'uA, rounded Id =', round(id_spec*2000000)/2, 'uA')
    id_w = mos.lookup('ID_W', GM_ID=gm_id_spec, L=L_spec, VDS=vgs, VSB=0)
    w = id_spec / id_w
    print('W =', round(w, 2), 'um, rounded W =', round(w*2)/2, 'um')      

In [ ]:

#NMOS Input Diffpairs
get_w(0,0.4398e-3,10,2,3.3/2,'GM')

V_GS = 0.818 V
gds = 923.92 nS
Cgg = 250.4 fF
f_T = 279.6 MHz
Id = 43.98 uA, rounded Id = 44.0 uA
42.711395498628335
W = 42.71 um, rounded W = 42.5 um


In [4]:
#PMOS Active Load
gm_id = 5
id = 44e-6
gm = gm_id*id
get_w(1,gm,5,2,3.3/2)

V_GS = 1.144 V
gds = 431.94 nS
Cgg = 293.7 fF
f_T = 119.2 MHz
Id = 44.0 uA, rounded Id = 44.0 uA
43.41722214305814
W = 43.42 um, rounded W = 43.5 um


In [5]:
#NMOS Tail Current Source
gm_id = 5
id = 2*44e-6
gm = gm_id*id
get_w(0,gm,5,2,3.3/2)

V_GS = 1.033 V
gds = 1156.61 nS
Cgg = 120.0 fF
f_T = 583.7 MHz
Id = 88.0 uA, rounded Id = 88.0 uA
18.829940763124256
W = 18.83 um, rounded W = 19.0 um


In [6]:
#PMOS Output Stage
gm_id = 10
gm = 0.44e-3*10
get_w(1,gm,10,2,3.3/2)

V_GS = 0.936 V
gds = 5204.14 nS
Cgg = 12170.9 fF
f_T = 57.5 MHz
Id = 440.0 uA, rounded Id = 440.0 uA
1877.261814380366
W = 1877.26 um, rounded W = 1877.5 um


In [9]:
#NMOS Ouput Stage
gm_id = 5
id = 440e-6
gm = gm_id*id
get_w(0,gm,5,2,3.3/2)

V_GS = 1.033 V
gds = 5783.03 nS
Cgg = 599.9 fF
f_T = 583.7 MHz
Id = 440.0 uA, rounded Id = 440.0 uA
94.14970381562128
W = 94.15 um, rounded W = 94.0 um


In [4]:
#Op-Amp Gain
gm_in = 0.4398e-3
gm_out = 0.4398e-3*10
gds_in1 = 923.92e-9
gds_in2 = 431.94e-9
gds_out1 = 5204.14e-9
gds_out2 = 5783.03e-9

DC_gain = gm_in*gm_out/((gds_in1+gds_in2)*(gds_out1+gds_out2))
print(f"DC gain = {np.log10(DC_gain)*20} dB")

DC gain = 102.26819514535018 dB
